In [ ]:
import pandas as pd
import numpy as np

def process_pair_probs(pair_prob_file, excel_file, output_file):
    # Load the pair probabilities data
    pair_probs = pd.read_csv(pair_prob_file)
    
    # Load the excel file with indices (assuming first column contains indices)
    indices_df = pd.read_excel(excel_file, header=None)
    indices = indices_df.iloc[:, 0].tolist()  # Get first column as list of indices
    
    # Create a dictionary to store unpaired probabilities
    unpaired_probs = {}
    paired_probs = {}
    
    # Process pair probabilities data
    for _, row in pair_probs.iterrows():
        i = row['i']
        j = row['j']
        prob = row['Probability']
        
        if j == -1:  # This is an unpaired probability
            unpaired_probs[i] = prob
        else:  # This is a pair probability
            if i not in paired_probs:
                paired_probs[i] = []
            paired_probs[i].append(prob)
    
    # Calculate paired probability for each nucleotide (1 - unpaired probability)
    all_probs = {}
    for i in unpaired_probs:
        all_probs[i] = 1 - unpaired_probs[i]
    
    # Prepare output data
    output_data = []
    
    for idx in indices:
        if idx not in all_probs:
            continue  # Skip if index not found
        
        # 1) Probability that the nucleotide is paired
        prob_paired = all_probs.get(idx, 0)
        
        # Helper function to calculate window averages
        def calculate_window_avg(start, window_size, direction='forward'):
            total = 0
            count = 0
            for offset in range(window_size):
                if direction == 'forward':
                    current_idx = start + offset
                else:  # backward
                    current_idx = start - offset
                
                if current_idx in all_probs:
                    total += all_probs[current_idx]
                    count += 1
            return total / count if count > 0 else 0
        
        # 2-5) Forward windows
        window_3nt = calculate_window_avg(idx, 3)
        window_6nt = calculate_window_avg(idx, 6)
        window_18nt = calculate_window_avg(idx, 18)
        window_36nt = calculate_window_avg(idx, 36)
        
        # 6-8) Backward windows from idx+35
        backward_start = idx + 35
        window_3nt_back = calculate_window_avg(backward_start, 3, 'backward')
        window_6nt_back = calculate_window_avg(backward_start, 6, 'backward')
        window_18nt_back = calculate_window_avg(backward_start, 18, 'backward')
        
        output_data.append([
            idx,
            prob_paired,
            window_3nt,
            window_6nt,
            window_18nt,
            window_36nt,
            window_3nt_back,
            window_6nt_back,
            window_18nt_back
        ])
    
    # Create output DataFrame
    output_df = pd.DataFrame(output_data, columns=[
        'Index',
        'Paired_Probability',
        'Avg_3nt_Window',
        'Avg_6nt_Window',
        'Avg_18nt_Window',
        'Avg_36nt_Window',
        'Avg_3nt_Window_Backward',
        'Avg_6nt_Window_Backward',
        'Avg_18nt_Window_Backward'
    ])
    
    # Save to CSV
    output_df.to_csv(output_file, index=False)
    print(f"Processed data saved to {output_file}")

# Example usage
pair_prob_file = 'pair_fraction.csv'
excel_file = 'mCH_on_off_rank.xlsx'
output_file = 'processed_pair_probabilities.csv'

process_pair_probs(pair_prob_file, excel_file, output_file)